# Analyse de la performance d’une stratégie GARP (Growth at a Reasonable Price)

**Master 2 Finance Internationale**

Objectif :
Analyser la performance d’une stratégie de conviction GARP, en décomposant l’alpha,
et en évaluant la persistance du couple rendement–risque face à des benchmarks mondiaux.


In [1]:
# =========================
# INSTALLATION DES PACKAGES
# =========================

! pip install pandas numpy yfinance matplotlib seaborn statsmodels openpyxl scikit-learn


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 54.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 90.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 74.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 81.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 79.0 MB/s  0:00:006m0:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15635 sha256=8d8f5dd7a1

## 1. Préparation des données

Cette section vise à importer, nettoyer et harmoniser les données nécessaires à l’analyse empirique :
- Actions du portefeuille GARP (Yahoo Finance)
- Benchmarks mondiaux (FactSet)
- Devise homogène : EUR
- Fréquence : mensuelle


In [3]:
# ======================================================
# 01. PRÉPARATION DES DONNÉES
# ======================================================
# - Import actions GARP (Yahoo Finance, prix ajustés)
# - Conversion en EUR
# - Construction portefeuille GARP équipondéré
# - Import benchmarks (FactSet - Excel)
# - Calcul rendements mensuels (log)
# - Contrôle & traitement des valeurs manquantes
# ======================================================

import pandas as pd
import numpy as np
import yfinance as yf
from pathlib import Path

# =========================
# PARAMÈTRES GÉNÉRAUX
# =========================

START_DATE = "2016-02-01"
END_DATE   = "2026-01-31"

DATA_RAW = Path("../data/raw")
DATA_PROCESSED = Path("../data/processed")
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# =========================
# UNIVERS GARP
# =========================

GARP_TICKERS = {
    "ADYEN.AS": "EUR",
    "SLYG.DE": "EUR",
    "AMZN": "USD",
    "META": "USD",
    "GOOGL": "USD",
    "CSU.TO": "CAD",
    "MELI": "USD",
    "DNP.WA": "PLN",
    "NU": "USD",
    "KNSL": "USD"
}

# =========================
# TAUX DE CHANGE (vs EUR)
# =========================

FX_TICKERS = {
    "USD": "EURUSD=X",
    "CAD": "EURCAD=X",
    "PLN": "EURPLN=X"
}

# =========================
# IMPORT ACTIONS (YAHOO)
# =========================

prices_local = pd.DataFrame()

for ticker, currency in GARP_TICKERS.items():
    data = yf.download(
        ticker,
        start=START_DATE,
        end=END_DATE,
        interval="1mo",
        auto_adjust=True,
        progress=False
    )["Close"].squeeze()

    data.name = ticker

    # Conversion en EUR si nécessaire
    if currency != "EUR":
        fx = yf.download(
            FX_TICKERS[currency],
            start=START_DATE,
            end=END_DATE,
            interval="1mo",
            auto_adjust=True,
            progress=False
        )["Close"].squeeze()

        data = data / fx

    prices_local[ticker] = data

prices_local = prices_local.dropna(how="all")

# =========================
# RENDEMENTS ACTIONS
# =========================

returns_stocks = np.log(prices_local / prices_local.shift(1)).dropna()

# =========================
# CONTRÔLE & TRAITEMENT DES NA (ACTIONS)
# =========================

if returns_stocks.isna().sum().sum() > 0:
    print("⚠ Valeurs manquantes détectées dans les rendements actions")
    returns_stocks = returns_stocks.apply(
        lambda x: x.fillna(x.median()),
        axis=0
    )
    print("✔ NA remplacés par la médiane de chaque actif")
else:
    print("✔ Aucun NA détecté dans les rendements actions")

# =========================
# PORTEFEUILLE GARP ÉQUIPONDÉRÉ
# =========================

returns_garp = returns_stocks.mean(axis=1)
returns_garp.name = "GARP"

# =========================
# IMPORT BENCHMARKS (FACTSET)
# =========================

def load_factset_index(filename, col_name):
    df = pd.read_excel(DATA_RAW / filename)
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df.set_index(df.columns[0], inplace=True)
    returns = np.log(df / df.shift(1)).dropna()
    returns.columns = [col_name]
    return returns

benchmarks = pd.concat([
    load_factset_index("msci_world_factset.xlsx", "MSCI_WORLD"),
    load_factset_index("msci_world_growth_factset.xlsx", "MSCI_WORLD_GROWTH"),
    load_factset_index("msci_world_value_factset.xlsx", "MSCI_WORLD_VALUE"),
    load_factset_index("ftse_all_world_factset.xlsx", "FTSE_ALL_WORLD")
], axis=1)

# =========================
# CONTRÔLE & TRAITEMENT DES NA (BENCHMARKS)
# =========================

if benchmarks.isna().sum().sum() > 0:
    print("⚠ Valeurs manquantes détectées dans les benchmarks")
    benchmarks = benchmarks.apply(
        lambda x: x.fillna(x.median()),
        axis=0
    )
    print("✔ NA remplacés par la médiane de chaque benchmark")
else:
    print("✔ Aucun NA détecté dans les benchmarks")

# =========================
# ALIGNEMENT FINAL
# =========================

returns_all = pd.concat(
    [returns_garp, benchmarks],
    axis=1
).dropna()

# =========================
# EXPORT
# =========================

returns_all.to_csv(DATA_PROCESSED / "monthly_returns_garp_benchmarks.csv")

print("✔ Data preparation completed successfully")
print(f"✔ Observations: {returns_all.shape[0]}")
print(f"✔ Period: {returns_all.index.min().date()} → {returns_all.index.max().date()}")


✔ Aucun NA détecté dans les rendements actions
⚠ Valeurs manquantes détectées dans les benchmarks
✔ NA remplacés par la médiane de chaque benchmark
✔ Data preparation completed successfully
✔ Observations: 0
✔ Period: NaT → NaT


/tmp/ipykernel_6344/241679935.py:129: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  benchmarks = pd.concat([
/tmp/ipykernel_6344/241679935.py:154: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  returns_all = pd.concat(
